In [1]:
import os
os.chdir("..")
os.getcwd()

'/Users/laurasisson/odor-pair'

In [2]:
import json
with open("dataset/single_fold.json") as f:
    split_data = json.load(f)
{k:len(v) for k, v in split_data.items()}

{'train': 43992, 'test': 39554, 'covered_notes': 77}

In [3]:
import analysis.fingerprint

mfpgen_big = analysis.fingerprint.make_mfpgen(radius=4,fpSize=2048)
mfpgen_small = analysis.fingerprint.make_mfpgen(radius=2,fpSize=1024)

In [4]:
train_mols = set()
for d in split_data["train"]:
    train_mols.update(d["edge"])

test_mols = set()
for d in split_data["test"]:
    test_mols.update(d["edge"])

len(train_mols), len(test_mols)

(1471, 1468)

In [5]:
all_blend_notes = {'oily', 'ethereal', 'fermented', 'bitter', 'soapy', 'phenolic', 'winey', 'roasted', 'spicy', 'fusel', 'tropical', 'anise', 'honey', 'aromatic', 'meaty', 'fresh', 'woody', 'melon', 'mentholic', 'clean', 'camphoreous', 'nutty', 'herbal', 'jammy', 'earthy', 'vegetable', 'caramellic', 'coconut', 'orris', 'bready', 'citrus', 'chemical', 'burnt', 'dairy', 'cheesy', 'fatty', 'floral', 'fruity', 'green', 'marine', 'coumarinic', 'licorice', 'mossy', 'tonka', 'creamy', 'waxy', 'animal', 'acidic', 'brown', 'cocoa', 'chocolate', 'sweet', 'rummy', 'sour', 'balsamic', 'coffee', 'solvent', 'fungal', 'berry', 'amber', 'cooling', 'onion', 'buttery', 'estery', 'powdery', 'musk', 'aldehydic', 'medicinal', 'alliaceous', 'minty', 'vanilla', 'thujonic', 'sulfurous', 'musty'}
all_blend_notes = list(all_blend_notes)

In [6]:
from tqdm.notebook import tqdm
import numpy as np
import graph.utils

def make_notes_vectors(dataset,mfpgen,should_concat):
    mol_to_embed = dict()
    for m in train_mols.union(test_mols):
        try:
            mol_to_embed[m] = analysis.fingerprint.smiles_to_embed(mfpgen,m)
        except Exception as e:
            print(m)
            continue
            
        xs = []
    ys = []
    empty = 0
    for d in tqdm(dataset):
        blnd = graph.utils.canonize(d["blend_notes"])
        
        mol1, mol2 = d["edge"]
        if not mol1 in mol_to_embed or not mol2 in mol_to_embed:
            continue
        
        if should_concat:
            x = np.concatenate([mol_to_embed[mol1],mol_to_embed[mol2]])
        else:
            x = mol_to_embed[mol1] + mol_to_embed[mol2]
        y = graph.utils.multi_hot(blnd,all_blend_notes)

        if x.sum() == 0 or y.sum() == 0:
            empty += 1
            continue

        xs.append(x)
        ys.append(y)
        
    print(f"Found {empty} empty blends.")        
    return np.array(xs), np.array(ys)

train_big_x, train_big_y = make_notes_vectors(split_data["train"],mfpgen_big,True)
test_big_x, test_big_y = make_notes_vectors(split_data["test"],mfpgen_big,True)
train_big_x.shape, test_big_x.shape, train_big_y.shape, test_big_y.shape

InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
CC(=O)c1ccc(C)n1
(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC


[11:51:58] SMILES Parse Error: syntax error while parsing: InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
[11:51:58] SMILES Parse Error: Failed parsing SMILES 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3' for input: 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3'
[11:51:59] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8
[11:51:59] SMILES Parse Error: syntax error while parsing: (C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC
[11:51:59] SMILES Parse Error: Failed parsing SMILES '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC' for input: '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC'


  0%|          | 0/43992 [00:00<?, ?it/s]

Found 262 empty blends.
InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
CC(=O)c1ccc(C)n1
(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC


[11:52:00] SMILES Parse Error: syntax error while parsing: InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
[11:52:00] SMILES Parse Error: Failed parsing SMILES 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3' for input: 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3'
[11:52:00] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8
[11:52:00] SMILES Parse Error: syntax error while parsing: (C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC
[11:52:00] SMILES Parse Error: Failed parsing SMILES '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC' for input: '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC'


  0%|          | 0/39554 [00:00<?, ?it/s]

Found 286 empty blends.


((43721, 4096), (39237, 4096), (43721, 74), (39237, 74))

In [7]:
import torch
import numpy as np
import scipy
import sklearn
import sklearn.linear_model
import sklearn.ensemble
import sklearn.svm
import sklearn.metrics
import sklearn.multioutput

class LogitRegression:
    EPS = 1e-5

    def __init__(self, model):
        self.model = model

    def _clip01(self, arr):
        return np.asarray(arr).clip(self.EPS, 1 - self.EPS)

    def fit(self, x, p):
        p = self._clip01(p)
        y = scipy.special.logit(p)
        return self.model.fit(x, y)

    def predict(self, x):
        y = self.model.predict(x)
        return scipy.special.expit(y)

def train_evaluate_model_logit(base_classifier,train_x,train_y,test_x,test_y):
    print("Training the model...")
    model = LogitRegression(base_classifier)
    model.fit(train_x, train_y)

    print("Predicting probabilities for AUROC calculation...")
    test_pred = model.predict(test_x)

     # Calculate AUROC using sklearn
    auc_micro_sklearn = sklearn.metrics.roc_auc_score(test_y, test_pred, average='micro')
    auc_macro_sklearn = sklearn.metrics.roc_auc_score(test_y, test_pred, average='macro')

    print("Sklearn Macro-averaged AUROC:", auc_macro_sklearn)
    
    auroc_per_label = {}
    for i, label in enumerate(all_blend_notes):
        auroc = sklearn.metrics.roc_auc_score(test_y[:, i], test_pred[:, i])
        auroc_per_label[label] = auroc
    auroc_per_label["TOTAL"] = auc_macro_sklearn

    return auroc_per_label

for should_concat in [True,False]:
    for size in ["big","small"]:
        for model_name in ["lr","rf"]:
            if size == "big":
                mfpgen = mfpgen_big
            else:
                mfpgen = mfpgen_small
                
            train_x, train_y = make_notes_vectors(split_data["train"],mfpgen,should_concat)
            test_x, test_y = make_notes_vectors(split_data["test"],mfpgen,should_concat)
            if model_name == "lr":
                model = sklearn.linear_model.LinearRegression()
            if model_name == "rf":
                model = sklearn.ensemble.RandomForestRegressor()
            if model_name == "svm":
                model = sklearn.multioutput.MultiOutputRegressor(sklearn.svm.SVR(kernel='linear'))

            print(f"Should Concat {should_concat}. Size {size}. Model {model_name}")
            results = train_evaluate_model_logit(model,train_x,train_y,test_x,test_y)
            with open(f"results/{should_concat}_{size}_{model_name}.json","w") as f:
                json.dump(results,f)

InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
CC(=O)c1ccc(C)n1
(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC


[11:52:02] SMILES Parse Error: syntax error while parsing: InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
[11:52:02] SMILES Parse Error: Failed parsing SMILES 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3' for input: 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3'
[11:52:02] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8
[11:52:02] SMILES Parse Error: syntax error while parsing: (C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC
[11:52:02] SMILES Parse Error: Failed parsing SMILES '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC' for input: '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC'


  0%|          | 0/43992 [00:00<?, ?it/s]

Found 262 empty blends.
InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
CC(=O)c1ccc(C)n1
(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC


[11:52:03] SMILES Parse Error: syntax error while parsing: InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
[11:52:03] SMILES Parse Error: Failed parsing SMILES 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3' for input: 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3'
[11:52:03] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8
[11:52:03] SMILES Parse Error: syntax error while parsing: (C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC
[11:52:03] SMILES Parse Error: Failed parsing SMILES '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC' for input: '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC'


...

  0%|          | 0/43992 [00:00<?, ?it/s]

Found 262 empty blends.
InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
CC(=O)c1ccc(C)n1
(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC


[12:44:07] SMILES Parse Error: syntax error while parsing: InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
[12:44:07] SMILES Parse Error: Failed parsing SMILES 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3' for input: 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3'
[12:44:07] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8
[12:44:07] SMILES Parse Error: syntax error while parsing: (C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC
[12:44:07] SMILES Parse Error: Failed parsing SMILES '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC' for input: '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC'


  0%|          | 0/39554 [00:00<?, ?it/s]

Found 286 empty blends.
Should Concat False. Size small. Model rf
Training the model...
Predicting probabilities for AUROC calculation...
Sklearn Macro-averaged AUROC: 0.661490916803318
